# Limit Order

The idea behind this example is we have a large market maker that has a large sell order and a relatively small buy order to purchase what the market maker is selling. This is supposed to be a "typical" example and we expect the ideal solution to take the price set by the market maker.

## Definitions

In [31]:
import collections

eth = int(1e18)

Trade = collections.namedtuple("Trade", "token amount")

class Order:
    def __init__(self, name, buy, sell, kind):
        self.name = name
        self.buy = buy
        self.sell = sell
        self.kind = kind
        
    def limit_price(self):
        if self.kind == "sell":
            a = self.buy
            b = self.sell
        else:
            a = self.sell
            b = self.buy
        return a.amount / b.amount
    
    def dump(self):
        if self.kind == "sell":
            a = self.sell
            b = self.buy
            action = "selling"
        else:
            a = self.buy
            b = self.sell
            action = " buying"
        print("{0:17} | {1} {2:24} {3} at a price of {4} {5}"
              .format(self.name, action,
                      a.amount, a.token,
                      self.limit_price(), b.token))

class Solution:
    def __init__(self, weth_price, filled = 1.0):
        self.prices = [
            1 * eth,
            
        ]

## Auction

In [32]:
### Tokens
tokens = [
    "DAI",
    "WETH",
]
fee_token = tokens[0]

### Orders

orders = [
    Order(
        name = "market maker (mm)",
        buy = Trade(token = "DAI", amount = 184_000 * eth),
        sell = Trade(token = "WETH", amount = 1_000 * eth),
        kind = "sell"
    ),
    Order(
        name = "buyer (b)",
        buy = Trade(token = "WETH", amount = 1 * eth),
        sell = Trade(token = "DAI", amount = 185 * eth),
        kind = "buy"
    ),
]

for order in orders:
    order.dump()

market maker (mm) | selling   1000000000000000000000 WETH at a price of 184.0 DAI
buyer (b)         |  buying      1000000000000000000 WETH at a price of 185.0 DAI


## Solution

A solution is defined as:

$$ \mathcal{S} = \{ \vec{p}, \vec{B} \} $$

Where $\vec{p} \in \mathbb{N}^T$ is a vector of prices ($T$ is the number of tokens) and $\vec{B}\in \mathbb{N}^O$ is a vector of *executed buy amounts* ($O$ is the number of orders).

Since we only have two tokens and two orders in our simple example, we can simplify this solution defnition to:

$$ \mathcal{S}_\text{simple} = \{p_\text{DAI}, p_\text{WETH}, B_\text{mm}, B_\text{b}\} $$

where $p_t$ is the price of token $t$ and $B_o$ is the *executed buy amount* of order $o$.

and furthermore, token conservation dictates that the sum of the *executed sell amounts* of non-fee tokens must equal the sum of *executed buy amounts* of non-fee tokens. Since we only have a non-fee token $\text{WETH}$ that means:

$$ S_\text{mm} = B_\text{b} $$

where $S_mm$ is the *executed sell amount* of order $mm$.

Additionally, we have the relation:

$$ B_o \cdot p_{buy_o} = S_o \cdot p_{sell_o} \cdot \tfrac{f_d - 1}{f_d} $$

where $f_d$ is the *fee denominator* and for order $o$, ${S}_o$ is the *executed sell amount*, $p_{sell_o}$ is the price of the order's sell token, ${B}_o$ is the *executed buy amount*, and $p_{buy_o}$ is the price of the order's buy token.

Applying this to order $\text{mm}$:

$$ B_\text{mm} \cdot p_\text{DAI} = S_\text{mm} \cdot p_\text{WETH} \cdot \frac{f_d - 1}{f_d} $$

$$ B_\text{mm} \cdot p_\text{DAI} = B_\text{b} \cdot p_\text{WETH} \cdot \frac{f_d - 1}{f_d} $$

$$ B_\text{mm} = B_\text{b} \cdot p_\text{WETH} \cdot \frac{f_d - 1}{f_d \cdot p_\text{DAI}} $$

This means that our solution is actually:

$$ \mathcal{S}_\text{simple} = \{p_\text{DAI}, p_\text{WETH}, B_\text{b} \cdot p_\text{WETH} \cdot \frac{f_d - 1}{f_d \cdot p_\text{DAI}}, B_\text{b}\} $$

Note that for a solution to be valid the price of the fee token is fixed. This means we have:

$$ p_\text{DAI} = 1 \bf{eth} $$

Given $f_d = 1000$ and if we assume that we are filling the buyer's order completely, so $B_\text{b} = 1 {\bf{eth}}$:

$$ \mathcal{S}_\text{simple} = \{1 {\bf{eth}}, p_\text{WETH}, B_\text{b} \cdot p_\text{WETH} \cdot \frac{999}{1000 {\bf{eth}}}, B_\text{b}\} $$

Observe that the solution is entirely in function of the price of WETH and how much of the buyer's order gets filled.

In [79]:
class SimpleSolution:
    def __init__(self, p_weth, B_b):
        self.prices = {
            "DAI": 1 * eth,
            "WETH": p_weth,
        }
        self.exec_buy_amounts = [
            ((B_b * p_weth * 999) // 1000) // eth,
            B_b,
        ]
        self.exec_sell_amounts = [
            ((self.exec_buy_amounts[0] * eth * 1000) // 999) // p_weth,
            ((B_b * p_weth * 1000) // 999) // eth,
        ]
    
    def dump(self):
        print("S_simple = {{ p: [{}, {}],\n             B: [{}, {}],\n             S: [{}, {}] }}"
              .format(self.prices["DAI"], self.prices["WETH"],
                      self.exec_buy_amounts[0], self.exec_buy_amounts[1],
                      self.exec_sell_amounts[0], self.exec_sell_amounts[1]))
        
SimpleSolution(p_weth = 184_184_184_184_184_185_000, B_b = eth).dump()

S_simple = { p: [1000000000000000000, 184184184184184185000],
             B: [184000000000000000815, 1000000000000000000],
             S: [1000000000000000000, 184368552736921106106] }


## Objective Value

The solution objective value is defined as:

$$
\mathcal{O} = \sum_{i \in O}{U_i} - \sum_{i \in O}{{dU}_i} + \frac{\text{fees}}{2}
$$

where the *utility* $U$ and the *disregarded utility* $dU$ are defined as:

$$
U_o = \frac{(B_o \cdot s_o - S_o \cdot b_o) \cdot p_{buy_o}}{s_o}
$$

$$
dU_o = \frac{(p_{sell_o} \cdot s_o - b_o \cdot p_{buy_o}) \cdot (s_o - S_o)}{s_o}
$$

where, for order $o$, ${B}_o$ is the *executed buy amount*, $p_{buy_o}$ is the price of the order's buy token, ${b}_o$ is the *order buy amount*, ${S}_o$ is the *executed sell amount*, $p_{sell_o}$ is the price of the order's sell token, and ${s}_o$ is the *order sell amount*.

The utility of the market maker's order:

$$
U_\text{mm} = \frac{(B_\text{mm} \cdot s_\text{mm} - S_\text{mm} \cdot b_\text{mm}) \cdot p_{\text{DAI}}}{s_\text{mm}}
$$

$$
= \frac{(B_\text{b} \cdot p_\text{WETH} \cdot \frac{999}{1000 {\bf{eth}}} \cdot 1000 {\bf{eth}} - B_b \cdot 184000 {\bf{eth}}) \cdot 1 {\bf{eth}}}{1000 {\bf{eth}}}
$$

$$
= \frac{(B_\text{b} \cdot (p_\text{WETH} \cdot \frac{999}{1000 {\bf{eth}}} \cdot 1000 {\bf{eth}} - 184000  {\bf{eth}})) \cdot 1 {\bf{eth}}}{1000 {\bf{eth}}}
$$

$$
= \frac{(B_\text{b} \cdot (p_\text{WETH} \cdot \frac{999}{1000 {\bf{eth}}} \cdot 1000 {\bf{eth}} - 184000  {\bf{eth}}))}{1000}
$$

The disregarded utility of the market maker's order:

$$
dU_\text{mm} = \frac{(p_{\text{WETH}} \cdot s_\text{mm} - b_\text{mm} \cdot p_{\text{DAI}}) \cdot (s_\text{mm} - S_\text{mm})}{s_\text{mm}}
$$

$$
= \frac{(p_{\text{WETH}} \cdot 1000 {\bf{eth}} - 184000 {\bf{eth}} \cdot 1 {\bf{eth}}) \cdot (184000 {\bf{eth}} - B_\text{b})}{1000 {\bf{eth}}}
$$

$$
= \frac{1 {\bf{eth}} \cdot (p_{\text{WETH}} \cdot 1000 - 184000 {\bf{eth}}) \cdot (184000 {\bf{eth}} - B_\text{b})}{1000 {\bf{eth}}}
$$

$$
= \frac{(p_{\text{WETH}} \cdot 1000 - 184000 {\bf{eth}}) \cdot (184000 {\bf{eth}} - B_\text{b})}{1000}
$$

The executed sell amount for the buyer:

$$
S_\text{b} \cdot p_\text{DAI} \cdot \frac{f_d - 1}{f_d} = B_\text{b} \cdot p_\text{WETH}
$$

$$
S_\text{b} = B_\text{b} \cdot p_\text{WETH} \cdot \frac{f_d}{f_d \cdot p_\text{DAI}}
$$

$$
= B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999 {\bf{eth}}}
$$

The utility of the buyer's order:

$$
U_\text{b} = \frac{(B_\text{b} \cdot s_\text{b} - S_\text{b} \cdot b_\text{b}) \cdot p_{\text{DAI}}}{s_\text{b}}
$$

$$
= \frac{(B_\text{b} \cdot 185 {\bf{eth}} - B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999 {\bf{eth}}} \cdot 1 {\bf{eth}}) \cdot 1 {\bf{eth}}}{185 {\bf{eth}}}
$$

$$
= \frac{(B_\text{b} \cdot 185 {\bf{eth}} - B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999}) \cdot 1 {\bf{eth}}}{185 {\bf{eth}}}
$$

$$
= \frac{(B_\text{b} \cdot 185 {\bf{eth}} - B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999})}{185}
$$

The disregarded utility of the buyer's order:

$$
dU_\text{b} = \frac{(p_{\text{DAI}} \cdot s_\text{b} - b_\text{b} \cdot p_{\text{WETH}}) \cdot (s_\text{b} - S_\text{b})}{s_\text{b}}
$$

$$
= \frac{(1 {\bf{eth}} \cdot 185 {\bf{eth}} - 1 {\bf{eth}} \cdot p_{\text{WETH}}) \cdot (185 {\bf{eth}} - B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999 {\bf{eth}}})}{185 {\bf{eth}}}
$$

$$
= \frac{1 {\bf{eth}} \cdot (185 {\bf{eth}} - p_{\text{WETH}}) \cdot (185 {\bf{eth}} - B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999 {\bf{eth}}})}{185 {\bf{eth}}}
$$

$$
= \frac{(185 {\bf{eth}} - p_{\text{WETH}}) \cdot (185 {\bf{eth}} - B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999 {\bf{eth}}})}{185}
$$

The total fees:

$$
\text{fees} = S_\text{b} - B_\text{mm}
$$

$$
= B_\text{b} \cdot p_\text{WETH} \cdot \frac{1000}{999 {\bf{eth}}} -  B_\text{b} \cdot p_\text{WETH} \cdot \frac{999}{1000 {\bf{eth}}}
$$

$$
= B_\text{b} \cdot p_\text{WETH} \cdot (\frac{1000}{999 {\bf{eth}}} - \frac{999}{1000 {\bf{eth}}})
$$